In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, cohen_kappa_score, classification_report, confusion_matrix, roc_auc_score, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

seed = 42

In [2]:
# Carregar os dados
df = pd.read_csv('../data/weaving_rejection_dataset_updated.csv')

In [3]:
# Separar features e target
X = df.drop('Rejection', axis=1)  # Features
y = df['Rejection']               # Target

In [4]:

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

In [5]:
# Escalar os dados (opcional, mas pode ser importante para alguns modelos)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
# Função para calcular a accuracy, Cohen's Kappa, AUC e R²
def evaluate_model_with_auc_and_r2(model, X_test, y_test):
    accuracy = model.score(X_test, y_test)
    y_pred = model.predict(X_test)
    kappa = cohen_kappa_score(y_test, y_pred)
    
    # Verifica se o modelo pode retornar probabilidades para calcular a AUC
    if hasattr(model, "predict_proba"):
        y_pred_proba = model.predict_proba(X_test)
        
        # Verifica se o problema é binário ou multiclasse
        if y_pred_proba.shape[1] == 2:
            # Caso binário
            auc = roc_auc_score(y_test, y_pred_proba[:, 1])  # Probabilidade da classe positiva
        else:
            # Caso multiclasse
            try:
                auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')  # One-vs-Rest para multiclasse
            except ValueError as e:
                print(f"Erro ao calcular AUC para multiclasse: {e}")
                auc = None
    else:
        auc = None  # Se o modelo não tiver `predict_proba`, AUC não será calculada

    # Cálculo do R² (usado aqui apenas para o contexto binário)
    try:
        r2 = r2_score(y_test, y_pred_proba[:, 1] if auc is not None else np.zeros_like(y_test))
    except ValueError as e:
        print(f"Erro ao calcular R²: {e}")
        r2 = None

    return accuracy, kappa, auc, r2



In [7]:
# Configuração do modelo de Random Forest sem PCA
cls_rf = RandomForestClassifier(random_state=seed, n_jobs=-1, n_estimators=300)

In [8]:
# Treinar o modelo 
cls_rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42)

In [9]:
# Avaliar o modelo sem PCA
accuracy, kappa, auc, r2 = evaluate_model_with_auc_and_r2(cls_rf, X_test, y_test)
print(f"\nAccuracy: {accuracy:.4f}")
print(f"Cohen's Kappa: {kappa:.4f}")
if auc is not None:
    print(f"AUC: {auc:.4f}")
else:
    print("AUC não pode ser calculada (o modelo não suporta 'predict_proba').")
if r2 is not None:
    print(f"R^2: {r2:.4f}")

Erro ao calcular AUC para multiclasse: Number of classes in y_true not equal to the number of columns in 'y_score'

Accuracy: 0.8489
Cohen's Kappa: 0.8305
AUC não pode ser calculada (o modelo não suporta 'predict_proba').
R^2: -0.0923


In [10]:
# PCA com 95% de variância explicada
pca = PCA(0.95)

In [11]:
# Ajustar o PCA e transformar os dados de treinamento
X_train_pca = pca.fit_transform(X_train_scaled)

In [12]:
# Transformar os dados de teste
X_test_pca = pca.transform(X_test_scaled)

In [13]:
# Ajustar o modelo com PCA
cls_rf_pca = RandomForestClassifier(random_state=seed, n_jobs=-1, n_estimators=300)
cls_rf_pca.fit(X_train_pca, y_train)

RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42)

In [15]:
# Avaliar o modelo com PCA
accuracy_pca, kappa_pca, auc_pca, r2_pca = evaluate_model_with_auc_and_r2(cls_rf_pca, X_test_pca, y_test)
print(f"\nAccuracy (com PCA): {accuracy_pca:.4f}")
print(f"Cohen's Kappa (com PCA): {kappa_pca:.4f}")
if auc_pca is not None:
    print(f"AUC (com PCA): {auc_pca:.4f}")
else:
    print("AUC não pode ser calculada (o modelo não suporta 'predict_proba').")
if r2_pca is not None:
    print(f"R^2 (com PCA): {r2_pca:.4f}")

Erro ao calcular AUC para multiclasse: Number of classes in y_true not equal to the number of columns in 'y_score'

Accuracy (com PCA): 0.8142
Cohen's Kappa (com PCA): 0.7923
AUC não pode ser calculada (o modelo não suporta 'predict_proba').
R^2 (com PCA): -0.0923


In [14]:
from sklearn.metrics import accuracy_score, cohen_kappa_score, r2_score
from sklearn.model_selection import KFold
import numpy as np

# Função para realizar a validação cruzada e calcular a média da acurácia, Cohen's Kappa e R²
def cross_validate_model(model, X, y, cv):
    accuracies = []
    kappas = []
    r2_scores = []
    
    kf = KFold(n_splits=cv, shuffle=True, random_state=42)
    
    # Convertendo X e y para arrays NumPy
    X = np.array(X)
    y = np.array(y)
    
    for train_index, test_index in kf.split(X):
        X_train_fold, X_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        
        model.fit(X_train_fold, y_train_fold)
        y_pred = model.predict(X_test_fold)
        
        acc = accuracy_score(y_test_fold, y_pred)
        kappa = cohen_kappa_score(y_test_fold, y_pred)
        
        # Calcular o R²
        if hasattr(model, "predict_proba"):
            y_pred_proba = model.predict_proba(X_test_fold)[:, 1]  # Probabilidade da classe positiva
            r2 = r2_score(y_test_fold, y_pred_proba)
        else:
            r2 = None  # Se o modelo não tiver `predict_proba`, R² não será calculado
        
        accuracies.append(acc)
        kappas.append(kappa)
        if r2 is not None:
            r2_scores.append(r2)
    
    return np.mean(accuracies), np.mean(kappas), np.mean(r2_scores) if r2_scores else None

# Cross-validation para 3, 5 e 10 folds
for cv in [3, 5, 10]:
    print(f"\nCross-validation com {cv} folds:")
    acc_mean, kappa_mean, r2_mean = cross_validate_model(cls_rf, X_train, y_train, cv=cv)
    print(f"Mean Accuracy: {acc_mean:.4f}")
    print(f"Cohen's Kappa: {kappa_mean:.4f}")
    if r2_mean is not None:
        print(f"Mean R²: {r2_mean:.4f}")
    else:
        print("R² não pode ser calculado (o modelo não suporta 'predict_proba').")



Cross-validation com 3 folds:


KeyboardInterrupt: 

In [16]:
# PCA com 95% de variância explicada
pca = PCA(0.95)

In [17]:
# Ajustar o PCA e transformar os dados de treinamento
X_train_pca = pca.fit_transform(X_train)

In [18]:
# Transformar os dados de teste
X_test_pca = pca.transform(X_test)

In [19]:
# Ajustar o modelo com PCA
cls_rf_pca = RandomForestClassifier(random_state=seed, n_jobs=-1, n_estimators=300)
cls_rf_pca.fit(X_train_pca, y_train)

KeyboardInterrupt: 

In [25]:
from sklearn.metrics import roc_auc_score

# Função para calcular a accuracy, Cohen's Kappa e AUC
def evaluate_model_with_auc(model, X_test, y_test):
    accuracy = model.score(X_test, y_test)
    y_pred = model.predict(X_test)
    kappa = cohen_kappa_score(y_test, y_pred)
    
    # Verifica se o modelo pode retornar probabilidades para calcular a AUC
    if hasattr(model, "predict_proba"):
        y_pred_proba = model.predict_proba(X_test)
        
        # Verifica se o problema é binário ou multiclasse
        if y_pred_proba.shape[1] == 2:
            # Caso binário
            auc = roc_auc_score(y_test, y_pred_proba[:, 1])  # Probabilidade da classe positiva
        else:
            # Caso multiclasse
            try:
                auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')  # One-vs-Rest para multiclasse
            except ValueError as e:
                print(f"Erro ao calcular AUC para multiclasse: {e}")
                auc = None
    else:
        auc = None  # Se o modelo não tiver `predict_proba`, AUC não será calculada

    return accuracy, kappa, auc

# Avaliar o modelo com PCA
accuracy_pca, kappa_pca, auc_pca = evaluate_model_with_auc(cls_rf_pca, X_test_pca, y_test)
print(f"\nAccuracy (com PCA): {accuracy_pca:.4f}")
print(f"Cohen's Kappa (com PCA): {kappa_pca:.4f}")
if auc_pca is not None:
    print(f"AUC (com PCA): {auc_pca:.4f}")
else:
    print("AUC não pode ser calculada (o modelo não suporta 'predict_proba').")


Erro ao calcular AUC para multiclasse: Number of classes in y_true not equal to the number of columns in 'y_score'

Accuracy (com PCA): 0.7040
Cohen's Kappa (com PCA): 0.6655
AUC não pode ser calculada (o modelo não suporta 'predict_proba').
